# 그래프

node와 node 사이에 연결된 edge의 정보를 가지고 있는 자료구조.

> 서로 다른 개체가 연결되어 있다는 이야기를 들으면 그래프 떠올리기

- 인접 행렬 `Adjacency Matrix` : 2차원 배열 사용
- 인접 리스트 `Adjacency List` : 리스트 사용

## 서로소 집합

Disjoint Sets. 공통 원소가 없는 두 집합을 의미함.

서로소 부분 집합들로 나누어진 원소들의 데이터를 처리하기 위한 자료구조임. 트리를 사용함.

<br>

**연산**

- 합집합 `union` : 2 개의 원소가 포함된 집합을 하나의 집합으로 함치는 연산
- 찾기 `find` : 특정한 원소가 속한 집합이 어떤 집합인지 알려주는 연산

<br>

**union 알고리즘**

각각 루트 노드를 찾아서, 더 큰 루트 노드가 더 작은 루트 노드를 가리키도록 함.

1. union 연산을 확인하여, 서로 연결된 두 노드 A, B를 확인함.
    (1) A와 B의 루트 노드 A', B'를 각각 찾음.
    (2) A'를 B'의 부모 노드로 설정함. (B'가 A'를 가리키도록)
2. 모든 union 연산을 처리할 때까지 1번 과정 반복

In [3]:
def findParent(parent, x):
    if parent[x] != x:
        return findParent(parent, parent[x])
    return x

In [4]:
def unionParent(parent, a, b):
    a = findParent(parent, a)
    b = findParent(parent, b)

    if a < b: parent[b] = a
    else: parent[a] = b

In [16]:
v, e = map(int, input().split())
parent = [0] * (v+1)

for i in range(1, v+1):
    parent[i] = i

In [14]:
for _ in range(e):
    a, b = map(int, input().split())
    unionParent(parent, a, b)

print('각 원소가 속한 집합:', end=' ')
for i in range(1, v+1):
    print(findParent(parent, i), end=' ')

print()

print('부모 테이블:', end=' ')
for i in range(1, v+1):
    print(parent[i], end=' ')

각 원소가 속한 집합: 1 1 1 1 5 5 
부모 테이블: 1 1 1 1 5 5 

하지만, 위의 find 함수는 매우 비효율적임.

최악의 경우 find 함수가 모든 노드를 다 확인하는 터라 시간 복잡도가 O(V)이다.

이런 find 함수를 경로 압축 `Path Compression` 기법을 적용하여 최적화 할 수 있다. **find 함수를 재귀적으로 호출한 뒤에, 부모 테이블값을 갱신한다.**

In [13]:
def findParent(parent, x):
    if parent[x] != x:
        parent[x] = findParent(parent, parent[x])
    return parent[x]

**cycle 판별**

서로소 집합은 다양한 알고리즘에 사용될 수 있음. 특히 서로소 집합은 무방향 그래프 내에서의 사이클을 판별할 때 사용할 수 있다는 특징이 있음.

1. 각 간선을 확인하며 두 노드의 루트 노드를 확인한다.
    (1) 루트 노드가 서로 다르다면 두 노드에 대하여 union 연산을 수행함.
    (2) 루트 노드가 서로 같다면 cycle이 발생한 것임.
2. 그래프에 포함되어 있는 모든 간선에 대하여 1번 과정 반복함.

In [17]:
cycle = False

for i in range(e):
    a, b = map(int, input().split())
    if findParent(parent, a) == findParent(parent, b):
        cycle = True
        break
    else:
        unionParent(parent, a, b)

print("cycle 발생" if cycle else "no cycle")

cycle 발생


## 신장 트리

하나의 그래프가 있을 때 모든 노드를 포함하면서 사이클이 존재하지 않는 부분 그래프를 말함.

> 모든 노드가 포함되어 서로 연결되면서 사이클이 존재하지 않는다는 조건은 트리의 성립 조건이기도 함.

<br>

**최소 신장 트리 알고리즘**

신장 트리 중에서 최소 비용으로 만들 수 있는 신장 트리를 찾는 알고리즘을 말함.

대표적인 알고리즘으로는 `크루스칼 알고리즘`이 있음.

<br>

**크루스칼 알고리즘**

Kruskal Algorithm. 그리디 알고리즘으로 분류됨. 서로소 집합 자료구조를 활용함.

1. 간선 데이터를 비용에 따라 오름차순으로 정렬함.
2. 간선을 하나씩 확인하며 현재의 간선이 사이클을 발생시키는지 확인함. (발생 시키면 안 됨)
3. 모든 간선에 대하여 2 번의 과정을 반복함.

즉, **거리가 가장 짧은 간선부터 차례대로 집합에 추가함. 다만, 사이클을 발생시키면 제외하고 연결함.**

In [1]:
v, e = map(int, input().split())
parent = [0] * (v+1)

edges = []
result = 0

for i in range(1, v+1):
    parent[i] = i

for _ in range(e):
    a, b, cost = map(int, input().split())
    edges.append((cost, a, b)) # cost로 정렬하기 위해 첫 번째 원소를 cost로

In [5]:
edges.sort()

for edge in edges:
    cost, a, b = edge

    if findParent(parent, a) != findParent(parent, b):
        unionParent(parent, a, b)
        result += cost

print(result)

159


## 위상 정렬

Topology Sort. 방향 그래프의 모든 노드를 방향성에 거스르지 않도록 순서대로 나열하는 것. 순서가 정해져 있는 일련의 작업을 차례대로 수행해야 할 때 사용함.

<br>

**알고리즘**

1. 진입차수가 0인 노드를 큐에 넣음.
    > 진입차수 `Indegree` : 특정한 노드로 들어오는 간선의 개수
2. 큐가 빌 때까지 다음의 과정을 반복함.
    (1) 큐에서 원소를 꺼내 해당 노드에서 출발하는 간선을 그래프에서 제거함.
    (2) 새롭게 진입차수가 0이 된 노드를 큐에 넣음.

<br>

**시간복잡도**

O(V+E)임. 차례대로 모든 노드를 확인하고, 해당 노드에서 출발하는 간선을 차례대로 제거해야 함.

In [1]:
from _collections import deque

In [2]:
v, e = map(int, input().split())
indegree = [0] * (v+1)
graph = [[] for _ in range(v+1)]

for _ in range(e):
    a, b = map(int, input().split())
    graph[a].append(b)
    indegree[b] += 1

In [5]:
def topologySort():
    result = []
    q = deque()

    for i in range(1, v+1):
        if indegree[i] == 0: q.append(i)

    while q:
        now = q.popleft()
        result.append(now)

        for i in graph[now]:
            indegree[i] -= 1

            if indegree[i] == 0: q.append(i)


    for i in result: print(i, end=' ')

In [6]:
topologySort()

1 2 5 3 6 4 7 